In [18]:
# Importing necessary libraries
import tkinter as Tk
from PIL import Image, ImageTk, ImageDraw, ImageFont
import os
import numpy as np
import collections
import csv
from random import randint
import pandas as pd

In [13]:
# Specify clip number 
clipNum = 'CLIP_011'

In [11]:
###
### Saves new frames with ovals overlayed on original frames 
### to show saved coordinate locations of multiple individuals/groups
### 

def saveVideo_MULT(sourceDir, destDir, positionsFile, verbose=True, start=1):
    df = pd.read_csv(positionsFile, header=None)
    df.dropna(inplace=True)
    colors = [(randint(0,255), randint(0,255), randint(0,255)) for x in df.index]
    font = ImageFont.truetype("VCR_OSD_MONO_1.001.ttf", 16)
    #font = ImageFont.load_default()
    for col in df.columns[start:]:
        if start == 0:
            imageNum = col + 1
        else: 
            imageNum = col
        
        if verbose: 
            if col % 10 == 0: print (col)
        path = os.path.join(sourceDir, 'frame_%04d.png' % (imageNum))
        newPath = os.path.join(destDir, 'frame_%04d.png' % (imageNum))

        img = Image.open(path, 'r')
        img = img.resize((1200,700))
        img_w, img_h = img.size
        background = Image.new('RGBA', (img_w, img_h), (255, 255, 255, 255))
        background.paste(img, (0,0))
        draw = ImageDraw.Draw(background)
        draw.text((20,20), str(col), (255,0,0), font=font)
        for ID, person in enumerate(df[col]):
            coords = [float(x) for x in person[1:-1].split(',')]
            draw.ellipse(coords, outline=colors[ID], width=2)
            
            # OPTIONAL: draw text on image with ID number for each individual/group
            draw.text((coords[0], coords[1]), str(ID), colors[ID], font=font)         
        background.save(newPath)

In [5]:
###
### Saves new frames with ovals toverlayed on original frames 
### Only shows locations for individuals tracked mulitple times
### Expects ID numbers for "good" trackings and "bad" trackings to verify which 
### duplicate trackings should be kept/thrown out
###

def saveVideo_DUPLICATES(sourceDir, destDir, positionsFile, goodSet, badSet, verbose=True):
    df = pd.read_csv(positionsFile, header=None)
    df.dropna(inplace=True)
    df.drop(df.columns[0], axis=1, inplace=True)
    # Using different colors to distinguish "good"/"bad" trackings
    red = (244, 80, 65)
    green = (47, 196, 39)
    #font = ImageFont.load_default()
    font = ImageFont.truetype("VCR_OSD_MONO_1.001.ttf", 15)
    for col in df.columns[1:]: 
        
        if verbose: 
            if col % 10 == 0: print (col)
        path = os.path.join(sourceDir, 'frame_%04d.png' % (col))
        newPath = os.path.join(destDir, 'frame_%04d.png' % (col))

        img = Image.open(path, 'r')
        img = img.resize((1200,700))
        img_w, img_h = img.size
        background = Image.new('RGBA', (img_w, img_h), (255, 255, 255, 255))
        background.paste(img, (0,0))
        draw = ImageDraw.Draw(background)
        draw.text((20,20), str(col), (255,0,0), font=font)
        for ID in goodSet:
            person = df[col][ID]
            coords = [float(x) for x in person[1:-1].split(',')]
            draw.ellipse(coords, outline=green, width=2)
            draw.text((coords[0], coords[1]), str(ID), green, font=font)
            
        for ID in badSet: 
            person = df[col][ID]
            coords = [float(x) for x in person[1:-1].split(',')]
            draw.ellipse(coords, outline=red, width=2)
            draw.text((coords[0], coords[1]), str(ID), red, font=font)
        
        background.save(newPath)

## tracked frames

In [ ]:
saveVideo_MULT('data/' + clipNum + '/FRAMES', 
               'data/' + clipNum + '/TRACKED_FRAMES', 
               'data/' + clipNum + '/DATA/positions.csv')

**command to generate video**  

`ffmpeg -framerate 10 -i data/CLIP_011/TRACKED_FRAMES/frame_%04d.png -b 5000k data/CLIP_011/VIDEO/ALL.mp4`

## duplicate frames

In [16]:
if not os.path.isdir('data/' + clipNum + '/DUPLICATES'):
    os.mkdir('data/' + clipNum + '/DUPLICATES')

In [ ]:
saveVideo_DUPLICATES('data/' + clipNum + '/FRAMES', 
                     'data/' + clipNum + '/DUPLICATES', 
                     'data/' + clipNum + '/DATA/positions.csv',
                     [26,50,64],
                     [25,49,62])

**command to generate video**  

`ffmpeg -framerate 10 -i data/CLIP_011/DUPLICATES/frame_%04d.png -b 5000k data/CLIP_011/VIDEO/DUPLICATES.mp4`

## clusters

In [17]:
if not os.path.isdir('data/' + clipNum + '/CLUSTERED_FRAMES'):
    os.mkdir('data/' + clipNum + '/CLUSTERED_FRAMES')

In [ ]:
saveVideo_MULT('data/' + clipNum + '/FRAMES', 
               'data/' + clipNum + '/CLUSTERED_FRAMES', 
               'data/' + clipNum + '/DATA/clusters_positions.csv',
                start=0)

**command to generate video**  

`ffmpeg -framerate 10 -i data/CLIP_011/CLUSTERED_FRAMES/frame_%04d.png -b 5000k data/CLIP_011/VIDEO/CLUSTERS.mp4`